#### Applications of pretrained models in the <span style="color:blue"> Hugging Face Transformers</span> library for sentiment classification. 

In [ ]:
from datasets import load_dataset
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd

**Loading the [yelp_review_full](https://huggingface.co/datasets/yelp_review_full) dataset**

In [ ]:
yelp_df = load_dataset("yelp_review_full")

**Selecting and inspecting data**

In [ ]:
print(yelp_df)

In [ ]:
print(type(yelp_df))

In [ ]:
print(yelp_df["train"])

In [ ]:
print(type(yelp_df["train"]))

In [ ]:
print(yelp_df["train"][0])

In [ ]:
print(yelp_df.num_columns, yelp_df.num_rows)

In [ ]:
print(yelp_df.shape)

In [ ]:
print(yelp_df.unique("label"))

**Transforming to other formats**

In [ ]:
yelp_df.set_format(type="pandas")

In [ ]:
print(yelp_df["train"])

In [ ]:
print(type(yelp_df["test"][:]))

In [ ]:
classifier = pipeline(task="text-classification", max_length=512)
sentiments = classifier(yelp_df["test"]["text"][0])

In [ ]:
pd.DataFrame(sentiments)

In [ ]:
train_df = yelp_df["test"][:10]
train_df

In [ ]:
print(train_df["text"])

In [ ]:
def sentiment_classification(text):
    outputs = classifier(text)
    label = outputs[0]["label"]
    return label


train_df["sentiment"] = train_df["text"].map(sentiment_classification)

In [ ]:
train_df

**Tokenization**

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
print(train_df["text"][5])
print(tokenizer.encode(train_df["text"][5]))
example_tokens = tokenizer.encode(
    train_df["text"][5], truncation=True, padding=True, add_special_tokens=True
)
print(tokenizer.decode(example_tokens))

In [ ]:
input_ids = torch.tensor(example_tokens).unsqueeze(0)
print(input_ids)

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=5)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
train_tokens = [
    tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)
    for text in train_df["text"]
]
ids = [torch.tensor(tokens).unsqueeze(0) for tokens in train_tokens]
print(ids)

In [ ]:
outputs = [model(input_ids=id) for id in ids]
predicted_labels = [torch.argmax(output.logits, dim=1) for output in outputs]
token_labels = [predicted_label[0].tolist() for predicted_label in predicted_labels]
train_df["new_label"] = pd.Series(token_labels)
train_df